In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from tqdm import tqdm

In [3]:
df = pd.read_csv("C:/Users/Aria/Desktop/M5 Capstone Project/Household Dataset/sales_train_validation.csv")
cal_df = pd.read_csv("C:/Users/Aria/Desktop/M5 Capstone Project/Household Dataset/calendar.csv")
price_df = pd.read_csv("C:/Users/Aria/Desktop/M5 Capstone Project/Household Dataset/sell_prices.csv")

In [4]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,0,0,0,...,1,0,1,1,1,0,0,0,0,1
1,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1,1,2,4,...,0,0,0,0,0,1,0,0,0,1
2,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,1,2,1,0,0,0,1,0
3,HOUSEHOLD_1_004_CA_1_validation,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,4,0,8,11,...,0,5,0,0,0,0,0,0,1,0
4,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,3,2,2,...,1,0,3,1,2,0,0,1,0,1


In [5]:
cal_df["d"]=cal_df["d"].apply(lambda x: int(x.split("_")[1]))
price_df["id"] = price_df["item_id"] + "_" + price_df["store_id"] + "_validation"

In [6]:
for day in tqdm(range(1886, 1914)):
    wk_id = list(cal_df[cal_df["d"]==day]["wm_yr_wk"])[0]
    wk_price_df = price_df[price_df["wm_yr_wk"]==wk_id]
    df = df.merge(wk_price_df[["sell_price", "id"]], on=["id"], how='inner')
    df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)]
    df.drop(columns=["sell_price"], inplace=True)

100%|█████████████████████████████████████████████| 28/28 [00:03<00:00,  9.10it/s]


In [7]:
df["dollar_sales"] = df[[c for c in df.columns if c.find("unit_sales")==0]].sum(axis=1)
df.drop(columns=[c for c in df.columns if c.find("unit_sales")==0], inplace=True)
df["weight"] = df["dollar_sales"] / df["dollar_sales"].sum()
df.drop(columns=["dollar_sales"], inplace=True)
df["weight"] /= 12

In [8]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,weight
0,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,0,0,0,...,0,1,1,1,0,0,0,0,1,3.938152e-06
1,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1,1,2,4,...,0,0,0,0,1,0,0,0,1,8.751448e-07
2,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,1,2,1,0,0,0,1,0,1.314916e-05
3,HOUSEHOLD_1_004_CA_1_validation,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,4,0,8,11,...,5,0,0,0,0,0,0,1,0,5.881208e-06
4,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,3,2,2,...,0,3,1,2,0,0,1,0,1,2.703787e-05


In [9]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))

In [10]:
df.dtypes

id           object
item_id      object
dept_id      object
cat_id       object
store_id     object
             ...   
d_1910        int64
d_1911        int64
d_1912        int64
d_1913        int64
weight      float64
Length: 1920, dtype: object

In [11]:
print('Largest sales value is', df[[c for c in df.columns if c.find('d_')==0]].max().max(), 
      '\nLargest int16 is', np.iinfo(np.int16).max)

Largest sales value is 626 
Largest int16 is 32767


In [12]:
for d in range(1914, 1942):
    df["d_" + str(d)] = np.nan

temp = df.drop(columns = [c for c in df.columns if c.find('d_')==0 and int(c.split('_')[1]) < 1100])

df_melted = temp.melt(id_vars=[n for n in temp.columns if n.find("id")!=-1],
       value_vars=[n for n in temp.columns if n.find("d_")==0],
       var_name = 'day', value_name = 'sales')

del temp

In [13]:
df_melted.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales
0,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1100,0.0
1,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1100,0.0
2,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1100,1.0
3,HOUSEHOLD_1_004_CA_1_validation,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1100,0.0
4,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1100,0.0


In [14]:
df_melted.shape

(8815740, 8)

In [15]:
df_melted["day"]=df_melted["day"].apply(lambda x: int(x.split("_")[1]))
df_melted=df_melted.merge(cal_df.drop(columns=["date", "wm_yr_wk", 
                                            "weekday"]), left_on=["day"], right_on=["d"]).drop(columns=["d"])

In [16]:
df_melted['event_name_1'].value_counts(dropna=False).head()

NaN              8082840
StPatricksDay      31410
SuperBowl          31410
Mother's day       31410
Pesach End         31410
Name: event_name_1, dtype: int64

In [17]:
df_melted['event_name_1'].astype('category').cat.codes.astype("int8").value_counts().head()

-1     8082840
 25      31410
 26      31410
 15      31410
 21      31410
dtype: int64

In [18]:
df_melted["event_name_1"]=df_melted["event_name_1"].astype('category').cat.codes.astype("int8")
df_melted["event_name_2"]=df_melted["event_name_2"].astype('category').cat.codes.astype("int8")
df_melted["event_type_1"]=df_melted["event_type_1"].astype('category').cat.codes.astype("int8")
df_melted["event_type_2"]=df_melted["event_type_2"].astype('category').cat.codes.astype("int8")

In [19]:
useful_ids = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
id_encodings = [id_col + '_encoding' for id_col in useful_ids]

for id_col in useful_ids:
    if id_col == 'item_id':
        df_melted[id_col + '_encoding'] = df_melted[id_col].astype('category').cat.codes.astype("int16")
    else: 
        df_melted[id_col + '_encoding'] = df_melted[id_col].astype('category').cat.codes.astype("int8")

In [20]:
df_melted.drop(columns=['year'] + useful_ids, inplace=True)
reduce_mem_usage(df_melted)

Mem. usage decreased to 294.26 Mb (57.3% reduction)


In [21]:
df_melted['test'] = df_melted[["id","sales"]].groupby("id")["sales"].shift(1).fillna(-1).astype(np.int16)
print(list(df_melted[df_melted['day']==1101]['test']) == list(df_melted[df_melted['day']==1100]['sales']))

True


In [22]:
df_melted.drop(columns=['test'], inplace=True)

In [23]:
# create lags, lags starts from 28 days ago to 77 days ago, spaced by 7 days
for lag in tqdm([28, 35, 42, 49, 56, 63, 70, 77]):
    df_melted["lag_" + str(lag)] = df_melted[["id","sales"]].groupby("id")["sales"].shift(lag).fillna(-1).astype(np.int16)

100%|██████████| 8/8 [00:15<00:00,  1.96s/it]


In [24]:
df_melted=df_melted[df_melted['lag_77']!=-1]
assert list(df_melted[df_melted['day']==1528]['lag_28']) == list(df_melted[df_melted['day']==1500]['sales'])

In [25]:
df_melted=df_melted.merge(cal_df[['d', 'wm_yr_wk']], left_on=['day'], right_on=['d']).drop(columns=['d'])

df_melted=df_melted.merge(price_df[['id', 'sell_price', 'wm_yr_wk']], on=['id', 'wm_yr_wk'], how='inner')

del price_df
del cal_df

In [26]:
df_melted.tail()

,id,day,sales,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,...,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,lag_77,wm_yr_wk,sell_price
7850596,HOUSEHOLD_2_514_WI_3_validation,1941,NaN,2,5,-1,-1,-1,-1,0,...,0,0,0,0,0,0,0,2,11617,19.54
7850597,HOUSEHOLD_2_515_WI_3_validation,1940,NaN,1,5,-1,-1,-1,-1,0,...,1,0,0,0,0,1,0,1,11617,1.97
7850598,HOUSEHOLD_2_515_WI_3_validation,1941,NaN,2,5,-1,-1,-1,-1,0,...,0,0,1,0,0,0,0,0,11617,1.97
7850599,HOUSEHOLD_2_516_WI_3_validation,1940,NaN,1,5,-1,-1,-1,-1,0,...,0,0,0,0,0,0,0,0,11617,5.94
7850600,HOUSEHOLD_2_516_WI_3_validation,1941,NaN,2,5,-1,-1,-1,-1,0,...,0,0,0,0,0,0,0,0,11617,5.94


In [27]:
best_params = {
            "objective" : "poisson",
            "metric" :"rmse",
            "force_row_wise" : True,
            "learning_rate" : 0.05,
    #         "sub_feature" : 0.8,
            "sub_row" : 0.75,
            "bagging_freq" : 1,
            "lambda_l2" : 0.1,
    #         "nthread" : 4
            "metric": ["rmse"],
        'verbosity': 1,
        'num_iterations' : 2048,
        'num_leaves': 64,
        "min_data_in_leaf": 50,
    }

In [28]:
X_train = df_melted[df_melted["day"] < 1886].drop(columns=["sales"])
X_val = df_melted[df_melted["day"].between(1886, 1913)].drop(columns=["sales"])
X_test = df_melted[df_melted["day"] > 1913].drop(columns=["sales"])

y_train = df_melted[df_melted["day"] < 1886]["sales"]
y_val = df_melted[df_melted["day"].between(1886, 1913)]["sales"]

In [29]:
%who DataFrame

X_test	 X_train	 X_val	 df	 df_melted	 wk_price_df	 


In [30]:
del wk_price_df
del df_melted

X_train.columns

Index(['id', 'day', 'wday', 'month', 'event_name_1', 'event_type_1',
       'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'item_id_encoding', 'dept_id_encoding', 'cat_id_encoding',
       'store_id_encoding', 'state_id_encoding', 'lag_28', 'lag_35', 'lag_42',
       'lag_49', 'lag_56', 'lag_63', 'lag_70', 'lag_77', 'wm_yr_wk',
       'sell_price'],
      dtype='object')

In [31]:
%%time

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.drop(columns=['id']).loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=id_encodings, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.drop(columns=['id']).loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=id_encodings,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

CPU times: user 5.59 s, sys: 468 ms, total: 6.06 s
Wall time: 6.06 s


In [32]:
%%time

m_lgb = lgb.train(best_params, train_data, valid_sets = [fake_valid_data], verbose_eval=1) 

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's rmse: 2.08388
[2]	valid_0's rmse: 2.05818
[3]	valid_0's rmse: 2.03376
[4]	valid_0's rmse: 2.01042
[5]	valid_0's rmse: 1.98814
[6]	valid_0's rmse: 1.96684
[7]	valid_0's rmse: 1.94634
[8]	valid_0's rmse: 1.92678
[9]	valid_0's rmse: 1.90783
[10]	valid_0's rmse: 1.88975
[11]	valid_0's rmse: 1.87244
[12]	valid_0's rmse: 1.8556
[13]	valid_0's rmse: 1.83938
[14]	valid_0's rmse: 1.82394
[15]	valid_0's rmse: 1.80918
[16]	valid_0's rmse: 1.79499
[17]	valid_0's rmse: 1.78138
[18]	valid_0's rmse: 1.76797
[19]	valid_0's rmse: 1.75518
[20]	valid_0's rmse: 1.74305
[21]	valid_0's rmse: 1.7313
[22]	valid_0's rmse: 1.72013
[23]	valid_0's rmse: 1.7092
[24]	valid_0's rmse: 1.69888
[25]	valid_0's rmse: 1.68911
[26]	valid_0's rmse: 1.67946
[27]	valid_0's rmse: 1.67025
[28]	valid_0's rmse: 1.66128
[29]	valid_0's rmse: 1.6527
[30]	valid_0's rmse: 1.64441
[31]	valid_0's rmse: 1.63656
[32]	valid_0's rmse: 1.62897
[33]	valid_0's rmse: 1.62173
[34]	valid_0's rmse: 1.61484
[35]	valid_0's rmse: 1.6082

In [33]:
for d in range(1886, 1914):
    df['F_' + str(d)] = m_lgb.predict(X_val[X_val['day']==d].drop(columns=['id']))

In [48]:
pred = df[[c for c in df.columns if c.find("d_") != 0]]
pred.head()

,id,item_id,dept_id,cat_id,store_id,state_id,weight,F_1886,F_1887,F_1888,...,F_1904,F_1905,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913
0,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3.938152e-06,0.407685,0.210528,0.358847,...,0.514922,0.892918,0.573229,0.394629,0.446405,0.226604,0.223855,0.564051,0.845198,0.647628
1,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,8.751448e-07,0.265195,0.229738,0.238446,...,0.288137,0.363080,0.340300,0.255412,0.230620,0.230962,0.234894,0.287701,0.353768,0.340300
2,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1.314916e-05,0.691016,0.496550,0.550160,...,0.595065,0.918465,0.764306,0.851881,0.534313,0.567553,0.515176,0.627116,0.929241,0.758270
3,HOUSEHOLD_1_004_CA_1_validation,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,5.881208e-06,1.567638,1.358925,1.606377,...,1.920667,2.387766,1.778684,1.483279,1.447529,1.529671,1.006851,1.816126,2.224827,1.768100
4,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,2.703787e-05,1.367518,1.558584,1.238576,...,1.187574,1.835976,1.796411,1.274156,1.634049,1.330256,1.133320,1.385928,1.800678,1.592039


In [50]:
pred.to_csv("/content/drive/MyDrive/DATA7903 Captone Project/original_predictions.csv", index = False)